In [0]:
!pip install --upgrade tensorflow-gpu

     |████████████████████████████████| 380.8MB 40kB/s 


In [0]:
!pip install soyspacing
!pip install soynlp
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
!pip install konlpy
!pip install sentencepiece
!pip install bert-tensorflow
# kakao 형태소 분석기 
import os
os.chdir('/content')
!git clone https://github.com/kakao/khaiii.git
os.chdir('/content/khaiii')
os.mkdir('/content/khaiii/build')
os.chdir('/content/khaiii/build')
!cmake ..
!make all
!make resource
!make package_python
os.chdir('package_python')
!pip install  . 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Korean Morphing Test

In [0]:
from konlpy.tag import Mecab
from khaiii import KhaiiiApi

In [0]:
mec = Mecab()

In [0]:
mec.morphs('안녕 하세요')

['안녕', '하', '세요']

In [0]:
khai = KhaiiiApi()

In [0]:
for i in khai.analyze('안녕 하세요'):
  print(i)

안녕	안녕/MAG
하세요	하/VV + 시/EP + 어요/EC


## This document is for comparing "Doc2Vec" and "Siemese LSTM"

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras as keras


In [0]:
tf.__version__

'2.0.0'

In [0]:
keras.__version__

'2.2.4-tf'

In [0]:
data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/0.Projects/Text Similarity/KCC /train_data.xlsx')

In [0]:
data.head()

,name,size,unit,name.1,size.1,unit.1,is_similar
0,가설사무실,"발주처,감리사무실,현장사무실 ,화장실 등",식,가설사무실,"발주처,감리사무실,현장사무실 ,화장실 등",식,1
1,가설사무실,"발주처,감리사무실,현장사무실 ,화장실 등",식,가설경비실 (2.4*2.4 기준),NaN,월,1
2,가설사무실,"발주처,감리사무실,현장사무실 ,화장실 등",식,이동식화장실(FRP),NaN,EA,1
3,가설사무실,"발주처,감리사무실,현장사무실 ,화장실 등",식,EGI펜스,NaN,M2,0
4,가설사무실,"발주처,감리사무실,현장사무실 ,화장실 등",식,스틸방음벽(자주식),NaN,M2,0


In [0]:
data.isna().sum()

name              0
size           3483
unit           6154
name.1            0
size.1        10529
unit.1            9
is_similar        0
dtype: int64

In [0]:
data['size'].fillna(' ', inplace=True)
data['unit'].fillna(' ', inplace=True)
data['size.1'].fillna(' ', inplace=True)

In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45730 entries, 0 to 45729
Data columns (total 7 columns):
name          45730 non-null object
size          45730 non-null object
unit          45730 non-null object
name.1        45730 non-null object
size.1        45730 non-null object
unit.1        45721 non-null object
is_similar    45730 non-null int64
dtypes: int64(1), object(6)
memory usage: 2.4+ MB


In [0]:
data.iloc[:, 0:-1]= data.iloc[:, 0:-1].astype('str')

In [0]:
sent_1 = data['name'] + ', ' + data['size'] + ', ' + data['unit']
sent_2 = data['name.1'] + ', ' + data['size.1'] + ', ' + data['unit.1']
target = data['is_similar']

In [0]:
sent_1 = pd.DataFrame({"sentence_1":sent_1,
                       "sentence_2":sent_2,
                       "target":target})

In [0]:
train_data = sent_1.copy()

In [0]:
import gc
del sent_1
del data
gc.collect()

46022

In [0]:
train_data.head()

,sentence_1,sentence_2,target
0,"가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식","가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식",1
1,"가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식","가설경비실 (2.4*2.4 기준), , 월",1
2,"가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식","이동식화장실(FRP), , EA",1
3,"가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식","EGI펜스, , M2",0
4,"가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식","스틸방음벽(자주식), , M2",0


## Doc 2 Vec

In [0]:
courpus = pd.concat([train_data['sentence_1'], train_data['sentence_2']]).to_list()

In [0]:
courpus

['가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 '가설사무실, 발주처,감리사무실,현장사무실 ,화장실 등, 식',
 